Random State for reproduceability

In [1]:
dagstuhl_seed=23372

# Data

In [2]:
import numpy as np
import pandas as pd

In [3]:
df = pd.read_csv('../../data/preprocessed.csv')
df = df.loc[:, ~df.columns.str.contains('^Unnamed')]

df_train = pd.read_csv('../../data/preprocessed-train.csv')
df_train = df_train.loc[:, ~df_train.columns.str.contains('^Unnamed')]

df_test = pd.read_csv('../../data/preprocessed-test.csv')
df_test = df_test.loc[:, ~df_test.columns.str.contains('^Unnamed')]

df.tail()

,label,0,1,2,3,4,5,6,7,8,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
4306,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4307,inactive,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4308,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4309,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4310,inactive,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
y = df_train['label']
X = df_train.loc[:, df_train.columns != "label"]
X.to_numpy().shape

(3592, 1024)

# AutoML Libraries

In [5]:
time_min = 30
time_sec = time_min * 60

## H2O

In [6]:
import h2o
from h2o.automl import H2OAutoML
h2o.init(nthreads=-1, log_level="TRACE")

Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "17.0.8.1" 2023-08-24; OpenJDK Runtime Environment (build 17.0.8.1+1-Ubuntu-0ubuntu122.04); OpenJDK 64-Bit Server VM (build 17.0.8.1+1-Ubuntu-0ubuntu122.04, mixed mode, sharing)
  Starting server from /opt/conda/lib/python3.9/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmphji_lbqg
  JVM stdout: /tmp/tmphji_lbqg/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmphji_lbqg/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.42.0.3
H2O_cluster_version_age:,1 month
H2O_cluster_name:,H2O_from_python_unknownUser_hgafym
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.887 Gb
H2O_cluster_total_cores:,8
H2O_cluster_allowed_cores:,8
H2O_cluster_status:,"locked, healthy"


In [7]:
hf = h2o.H2OFrame(df_train)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [8]:
%%time
aml = H2OAutoML(max_runtime_secs=time_sec, seed=dagstuhl_seed, verbosity="debug")
aml.train(y = 'label' , training_frame = hf)

AutoML progress: |
18:18:00.855: Project: AutoML_1_20230922_181800
18:18:00.859: 5-fold cross-validation will be used.
18:18:00.861: Setting stopping tolerance adaptively based on the training frame: 0.016685216106649997
18:18:00.861: Build control seed: 23372
18:18:00.865: training frame: Frame key: AutoML_1_20230922_181800_training_Key_Frame__upload_8bfa346ee01820b397e31d91bab24bb2.hex    cols: 1025    rows: 3592  chunks: 4    size: 403868  checksum: -87072914012808
18:18:00.865: validation frame: NULL
18:18:00.865: leaderboard frame: NULL
18:18:00.866: blending frame: NULL
18:18:00.866: response column: label
18:18:00.867: fold column: null
18:18:00.867: weights column: null
18:18:00.895: Loading execution steps: [{XGBoost : [def_2 (1g, 10w), def_1 (2g, 10w), def_3 (3g, 10w), grid_1 (4g, 90w), lr_search (6g, 30w)]}, {GLM : [def_1 (1g, 10w)]}, {DRF : [def_1 (2g, 10w), XRT (3g, 10w)]}, {GBM : [def_5 (1g, 10w), def_2 (2g, 10w), def_3 (2g, 10w), def_4 (2g, 10w), def_1 (3g, 10w), grid_1 

key,value
Stacking strategy,cross_validation
Number of base models (used / total),4/6
# GBM base models (used / total),1/1
# XGBoost base models (used / total),1/1
# DRF base models (used / total),1/2
# GLM base models (used / total),1/1
# DeepLearning base models (used / total),0/1
Metalearner algorithm,GLM
Metalearner fold assignment scheme,Random
Metalearner nfolds,5


In [9]:
hf_test = h2o.H2OFrame(df_test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [10]:
from sklearn.metrics import accuracy_score
y_pred = aml.predict(hf_test)
accuracy_score(df_test["label"], y_pred.as_data_frame()['predict']) 

stackedensemble prediction progress: |███████████████████████████████████████████| (done) 100%


0.9443671766342142

In [11]:
lb = h2o.automl.get_leaderboard(aml, extra_columns = "ALL")
lb

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse,training_time_ms,predict_time_per_row_ms,algo
StackedEnsemble_BestOfFamily_4_AutoML_1_20230922_181800,0.81143,0.206638,0.976673,0.445431,0.235979,0.055686,379,0.049236,StackedEnsemble
StackedEnsemble_BestOfFamily_5_AutoML_1_20230922_181800,0.810435,0.207089,0.977956,0.452823,0.236443,0.0559052,557,0.080024,StackedEnsemble
StackedEnsemble_AllModels_4_AutoML_1_20230922_181800,0.807343,0.207821,0.97818,0.445281,0.236665,0.0560105,2276,0.153184,StackedEnsemble
StackedEnsemble_Best1000_1_AutoML_1_20230922_181800,0.806833,0.208537,0.978026,0.443808,0.23735,0.056335,2287,0.1998,StackedEnsemble
StackedEnsemble_BestOfFamily_7_AutoML_1_20230922_181800,0.806788,0.208316,0.976653,0.456369,0.236921,0.0561316,2697,0.059739,StackedEnsemble
StackedEnsemble_AllModels_3_AutoML_1_20230922_181800,0.806752,0.207665,0.977453,0.429174,0.236421,0.0558948,1858,0.137612,StackedEnsemble
StackedEnsemble_AllModels_6_AutoML_1_20230922_181800,0.805612,0.208929,0.97805,0.438039,0.237439,0.0563774,13554,0.203931,StackedEnsemble
StackedEnsemble_BestOfFamily_2_AutoML_1_20230922_181800,0.804071,0.20998,0.977521,0.447354,0.238091,0.0566873,570,0.043956,StackedEnsemble
StackedEnsemble_BestOfFamily_3_AutoML_1_20230922_181800,0.803668,0.210081,0.977622,0.422382,0.238159,0.0567196,482,0.04503,StackedEnsemble
StackedEnsemble_AllModels_1_AutoML_1_20230922_181800,0.803543,0.210387,0.977933,0.447354,0.238497,0.0568807,513,0.058383,StackedEnsemble


In [12]:
m = aml.leader
m.params.keys()

dict_keys(['model_id', 'training_frame', 'response_column', 'validation_frame', 'blending_frame', 'base_models', 'metalearner_algorithm', 'metalearner_nfolds', 'metalearner_fold_assignment', 'metalearner_fold_column', 'metalearner_params', 'metalearner_transform', 'max_runtime_secs', 'weights_column', 'offset_column', 'seed', 'score_training_samples', 'keep_levelone_frame', 'export_checkpoints_dir', 'auc_type'])

In [13]:
# Get AutoML event log
log = aml.event_log
log

timestamp,level,stage,message,name,value
18:18:00.855,INFO,Workflow,Project: AutoML_1_20230922_181800,,
18:18:00.859,INFO,Validation,5-fold cross-validation will be used.,,
18:18:00.861,INFO,Validation,Setting stopping tolerance adaptively based on the training frame: 0.016685216106649997,,
18:18:00.861,INFO,Validation,Build control seed: 23372,,
18:18:00.865,INFO,DataImport,training frame: Frame key: AutoML_1_20230922_181800_training_Key_Frame__upload_8bfa346ee01820b397e31d91bab24bb2.hex cols: 1025 rows: 3592 chunks: 4 size: 403868 checksum: -87072914012808,,
18:18:00.865,INFO,DataImport,validation frame: NULL,,
18:18:00.865,INFO,DataImport,leaderboard frame: NULL,,
18:18:00.866,INFO,DataImport,blending frame: NULL,,
18:18:00.866,INFO,DataImport,response column: label,,
18:18:00.867,INFO,DataImport,fold column: null,,


In [14]:
# Get training timing info
info = aml.training_info
info

{'creation_epoch': '1695406681',
 'start_epoch': '1695406681',
 'start_XGBoost_def_2': '1695406681',
 'start_GLM_def_1': '1695406693',
 'start_GBM_def_5': '1695406699',
 'start_StackedEnsemble_best_of_family_1': '1695406722',
 'start_XGBoost_def_1': '1695406723',
 'start_DRF_def_1': '1695406727',
 'start_GBM_def_2': '1695406741',
 'start_GBM_def_3': '1695406765',
 'start_GBM_def_4': '1695406790',
 'start_StackedEnsemble_best_of_family_2': '1695406812',
 'start_StackedEnsemble_all_2': '1695406812',
 'start_XGBoost_def_3': '1695406813',
 'start_DRF_XRT': '1695406821',
 'start_GBM_def_1': '1695406831',
 'start_DeepLearning_def_1': '1695406845',
 'start_StackedEnsemble_best_of_family_3': '1695406859',
 'start_StackedEnsemble_all_3': '1695406860',
 'start_XGBoost_grid_1': '1695406860',
 'start_GBM_grid_1': '1695407128',
 'start_DeepLearning_grid_1': '1695407356',
 'start_StackedEnsemble_best_of_family_4': '1695408115',
 'start_StackedEnsemble_all_4': '1695408115',
 'start_DeepLearning_grid_